In [33]:
import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt
import pylab as plb
import numpy as np
import keras

In [34]:
matfn='train.mat'
data=sio.loadmat(matfn)
train_y=data['trainX'][...,784]
train_x=data['trainX'][...,0:784]

labels=np.zeros(shape=(len(train_y),10))
for i in range(len(train_y)):
    labels[i,(train_y[i]-1)]=1
    
train_x=train_x/255


index=np.arange(len(train_x))
np.random.shuffle(index)
train_x=train_x[index]
labels=labels[index]
train_set=train_x[0:round(len(train_x)*0.8)]
train_label=labels[0:round(len(train_x)*0.8)]
valid_set=train_x[round(len(train_x)*0.8):]
valid_label=labels[round(len(train_x)*0.8):]

# BP Network - Tensorflow

In [35]:
learning_rate = 0.001
epochs = 5
batch_size = 80

In [36]:
hidden_1 = 256
hidden_2 = 256
n_input = 784
n_classes = 10
n_obs = train_set.shape[0]

In [56]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [38]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, hidden_1])),
    'h2': tf.Variable(tf.random_normal([hidden_1, hidden_2])),
    'out': tf.Variable(tf.random_normal([hidden_2, n_classes]))
}

In [39]:
biases = {
    'b1': tf.Variable(tf.random_normal([hidden_1])),
    'b2': tf.Variable(tf.random_normal([hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [40]:
def bp_network(x,weights,bias):
    
    # First Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x,weights['h1']),bias['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    # Second Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    # Last Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer
    

In [57]:
# predict initialize
pred = bp_network(x, weights, biases)

In [58]:
# choose loss function
loss = tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y)

In [59]:
cost = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [61]:
keep_prob = tf.placeholder("float")
correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
prediction=tf.argmax(pred,1)

In [62]:
# Launch the session
sess = tf.InteractiveSession()

# Intialize all the variables
init = tf.global_variables_initializer()
sess.run(init)

# Training Epochs
# Essentially the max amount of loops possible before we stop
# May stop earlier if cost/loss limit was set
for epoch in range(epochs):

    # Start with cost = 0.0
    avg_cost = 0.0

    # Convert total number of batches to integer
    total_batch = int(n_obs/batch_size)

    # Loop over all batches
    for i in range(total_batch):

        # Grab the next batch of training data and labels
        # batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_ind=np.random.randint(0,48000,50)
        batch=[train_set[batch_ind],train_label[batch_ind]]
        # Feed dictionary for optimization and loss value
        # Returns a tuple, but we only need 'c' the cost
        # So we set an underscore as a "throwaway"
        _, c = sess.run([optimizer, cost], feed_dict={x: batch[0], y: batch[1]})

        # Compute average loss
        avg_cost += c / total_batch
        if i%100 == 0:  
            train_accuracy = accuracy.eval(feed_dict={
                x:batch[0], y: batch[1], keep_prob: 1.0})  
            print ("step %d, training accuracy %.3f"%(i, train_accuracy))

    print("Epoch: {} cost={:.4f}".format(epoch+1,avg_cost))

print("Model has completed {} Epochs of Training".format(epochs))

step 0, training accuracy 0.120
step 100, training accuracy 0.620
step 200, training accuracy 0.780
step 300, training accuracy 0.840
step 400, training accuracy 0.660
step 500, training accuracy 0.920
Epoch: 1 cost=190.4891
step 0, training accuracy 0.860
step 100, training accuracy 0.920
step 200, training accuracy 0.920
step 300, training accuracy 0.980
step 400, training accuracy 0.960
step 500, training accuracy 0.880
Epoch: 2 cost=44.9615
step 0, training accuracy 0.880
step 100, training accuracy 0.900
step 200, training accuracy 0.860
step 300, training accuracy 0.900
step 400, training accuracy 0.920
step 500, training accuracy 0.920
Epoch: 3 cost=29.7683
step 0, training accuracy 0.980
step 100, training accuracy 0.880
step 200, training accuracy 0.880
step 300, training accuracy 0.920
step 400, training accuracy 0.840
step 500, training accuracy 0.880
Epoch: 4 cost=22.0613
step 0, training accuracy 0.960
step 100, training accuracy 0.940
step 200, training accuracy 0.920
ste

In [63]:
prediction.eval(feed_dict={x: valid_set, keep_prob: 1.0}, session=sess)

array([7, 8, 4, ..., 1, 4, 3])

In [47]:
accuracy.eval(feed_dict={
        x: train_set[0:5000], y: train_label[0:5000], keep_prob: 1.0})

0.93599999

In [51]:
def model_eval(y,model, X_test, Y_test):
    acc_value = tf.reduce_mean(keras.metrics.categorical_accuracy(y, pred))
    cur_acc = acc_value.eval(feed_dict={x: X_test,
                           y: Y_test,
                           keras.backend.learning_phase(): 0})
    return cur_acc


In [ ]:
# 待研究
def batch_eval()


In [64]:
# 正常的training accuracy
model_eval(y,pred,train_set,train_label)

0.93685418

### fast gradient sign method

In [48]:
grad, = tf.gradients(loss,x)

In [49]:
epsilon = 0.07

In [68]:
adv_x = tf.stop_gradient(x + epsilon * tf.sign(grad))

In [ ]:
###### batch_eval 函数解构（没完）
#eval_params = {'batch_size': FLAGS.batch_size}
X_test_adv, = batch_eval(sess, [x], [adv_x], [X_test])

In [ ]:
###model_eval 函数解构(部分完成） 找出X_test_adv的结果
# adv example 的 training accuracy
model_eval(y, pred, X_test_adv, Y_test) #正常完成batch_eval函数应该就能得到这个accuracy

# 对抗训练

In [ ]:
# 完成adv example的accuracy后，完成fast_gradient函数，然后下面进行对抗训练

In [53]:
pred_new = bp_network(x, weights, biases)

adv_x_new = fast_gradient(x, predictions_2, eps=0.3)
pred_adv = bp_network(adv_x, weights, biases)

model_eval(y,pred_new,train_set,train_label) 
model_eval(y,pred_adv,train_set,train_label) 

# 以下是备用

# Keras bp

In [24]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
model = Sequential()

model.add(Dense(units = 200, input_dim = 784, activation = 'tanh'))
model.add(Dense(units = 10, activation = 'sigmoid'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(train_set,train_label, epochs = 5, batch_size = 80) 


Using TensorFlow backend.


Epoch 1/5
48000/48000 [==============================] - 7s - loss: 0.3689 - acc: 0.9017     
Epoch 2/5
48000/48000 [==============================] - 5s - loss: 0.1644 - acc: 0.9532     
Epoch 3/5
48000/48000 [==============================] - 9s - loss: 0.1147 - acc: 0.9676     
Epoch 4/5
48000/48000 [==============================] - 5s - loss: 0.0868 - acc: 0.9754     
Epoch 5/5
48000/48000 [==============================] - 5s - loss: 0.0668 - acc: 0.9815     


In [65]:
# 可能有用的函数。。 可以等之前的code跑通了再把这些放到一起
def model_loss(y,model):
    op = model.op
    if "softmax" in str(op).lower():
        logits, = op.inputs
    else:
        logits = model
    out  = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y)
    return out

def fast_gradient(x,pred,epsilon):
    """
    :param x: the input placeholder
    :param pred: the model's output tensor
    :param epsilon: input variation parameter
    
    :return: a tensor for the adversarial example
    """
    
    # compute loss
    y = tf.to_float(tf.equal(pred,tf.reduce_max(pred,1,keep_dims = True)))
    y = y / tf.reduce_sum(y,1,keep_dims = True)
    
    loss = model_loss(y,pred)
    # compute gradient
    
    grad, = tf.gradients(loss,x)
    
    adv_x = tf.stop_gradient(x + epsilon * tf.sign(grad))
    
    return adv_x

In [67]:
adv = fast_gradient(x,pred,epsilon=0.07)